In [1]:
import sys
import nltk
# nltk.download('wordnet')
# nltk.download('stopwords')

In [2]:
sys.path.append('../../src/util')
import ouraws
import ourgraphs
import textutil

In [3]:
BUCKET_NAME = "collegier"
DATA_DIR = "data"

In [4]:
import boto3

def list_parquet_files(bucket_name, folder_path=''):
    s3 = boto3.client('s3')
    paginator = s3.get_paginator('list_objects_v2')
    parquet_files = []
    
    for page in paginator.paginate(Bucket=bucket_name, Prefix=folder_path):
        for obj in page['Contents']:
            if obj['Key'].endswith('.parquet'):
                parquet_files.append(obj['Key'])
    
    return parquet_files

In [5]:
files = list_parquet_files(bucket_name=BUCKET_NAME, folder_path=DATA_DIR)
len(files)

469

In [6]:
toplevelfiles = [d for d in files if not d.startswith('data/data/') and not d.startswith('data/archive')]
len(toplevelfiles)

49

In [7]:
import s3fs
import pandas as pd

def load_parquet_from_s3(bucket, key):
    fs = s3fs.S3FileSystem()

    # Formulate the full path
    path = f"{bucket}/{key}"

    # Load the file directly into a pandas DataFrame
    df = pd.read_parquet(fs.open(path))

    return df

In [8]:
austin = load_parquet_from_s3(BUCKET_NAME, "data/austin-opinion-SNAPSHOT.parquet")
austin.shape

(5713, 6)

In [9]:
austin.head()

,title,url,body,year,month,day
0,\n\t\t\t\t\t\t\t\t\t\t\t\tThe Texan needs dive...,https://thedailytexan.com/2022/12/02/the-texan...,The Texan needs diverse political perspectives...,2022,12,2
1,\n\t\t\t\t\t\t\t\t\t\t\t\tWhat does free speec...,https://thedailytexan.com/2022/12/02/what-does...,What does free speech mean to you?\nNationwide...,2022,12,2
2,\n\t\t\t\t\t\t\t\t\t\t\t\tFree speech has cons...,https://thedailytexan.com/2022/12/02/free-spee...,Free speech has consequences\nEditor’s note: T...,2022,12,2
3,\n\t\t\t\t\t\t\t\t\t\t\t\tLiberum oratio\t\t\t...,https://thedailytexan.com/2022/12/02/liberum-o...,"Liberum oratio\nTo quote the Father of Texas, ...",2022,12,2
4,\n\t\t\t\t\t\t\t\t\t\t\t\tWhen to say what is ...,https://thedailytexan.com/2022/12/02/when-to-s...,When to say what is not to be said\nEditor’s n...,2022,12,2


In [10]:
austin_polarity = load_parquet_from_s3(BUCKET_NAME, "data/austin-POLARITY.parquet")
austin_polarity.shape

(7, 4)

In [11]:
austin_polarity.head()

,year,article_count,polarity_sum,polarity_avg
0,2010,56,12.761207,0.227879
1,2011,385,-236.699265,-0.614803
2,2012,484,-141.983378,-0.293354
3,2013,514,-937.440895,-1.823815
4,2014,545,-524.639852,-0.962642


In [12]:
stanford_opinions = load_parquet_from_s3(BUCKET_NAME, "data/stanford-opinions-SNAPSHOT.parquet")
stanford_opinions.shape

(4914, 6)

In [13]:
type(stanford_opinions)

pandas.core.frame.DataFrame

In [14]:
stanford_opinions.columns

Index(['title', 'url', 'body', 'year', 'month', 'day'], dtype='object')

In [15]:
stanford_opinions.head(2)

,title,url,body,year,month,day
https://stanforddaily.com/2010/12/01/sense-and-nonsense-making-the-dream-a-reality/,Sense and Nonsense: Making the DREAM a Reality,https://stanforddaily.com/2010/12/01/sense-and...,Sense and Nonsense: Making the DREAM a Reality...,2010,12,1
https://stanforddaily.com/2010/12/01/the-transitive-property-the-best-birthday-present-ever/,The Transitive Property: The Best Birthday Pre...,https://stanforddaily.com/2010/12/01/the-trans...,The Transitive Property: The Best Birthday Pre...,2010,12,1


In [16]:
stanford_opinions['year'].value_counts()

year
2010    506
2011    482
2012    457
2014    455
2015    447
2017    427
2016    408
2020    379
2018    367
2013    340
2019    328
2021    187
2022    131
Name: count, dtype: int64